# 逆行列 (Inverse Matrix) の定義

逆行列とは、ある正方行列 $A$ に対して、掛けると単位行列 $I$ になるような行列のことです。
数に例えると、$5$ に対する $1/5$ （逆数）のような関係です。

### 数学的定義
$n \times n$ の正方行列 $A$ に対して、以下の関係を満たす行列 $B$ が存在するとき、$B$ を $A$ の**逆行列**と呼び、$A^{-1}$ と表記します。

$$AB = BA = I$$

ここで $I$ は単位行列（対角成分が1で、他が0の行列）です。

### 逆行列が存在する条件（正則行列）
すべての行列に逆行列があるわけではありません。逆行列を持つ行列を**正則行列**と呼び、以下の条件を満たす必要があります。
1. **正方行列**であること（行数と列数が同じ）。
2. **行列式 (Determinant)** が $0$ ではないこと（$\det(A) \neq 0$）。
3. **フルランク**であること（すべての行/列が独立していること）。

> **なぜ学ぶのか？**
> 機械学習の最小二乗法などの公式 $\mathbf{w} = (X^T X)^{-1} X^T y$ において、パラメータを解くために不可欠な操作だからです。

In [1]:
import numpy as np

# 行列 A の定義
A = np.array([[1, 2],
              [3, 4]])

print("行列 A:")
print(A)

# 1. NumPyのメソッドで逆行列を求める
A_inv = np.linalg.inv(A)

print("\n逆行列 A_inv (np.linalg.inv):")
print(A_inv)

# 2. 検証: A @ A_inv が単位行列になるか？
identity_check = A @ A_inv
print("\n検証 (A @ A_inv):")
print(np.round(identity_check, 10)) # 小数点以下の誤差を丸めて表示

行列 A:
[[1 2]
 [3 4]]

逆行列 A_inv (np.linalg.inv):
[[-2.   1. ]
 [ 1.5 -0.5]]

検証 (A @ A_inv):
[[1. 0.]
 [0. 1.]]


# 逆行列の仕組み：掃き出し法（ガウス・ジョルダン法）

NumPyの内部で何が行われているのかを理解するために、**掃き出し法**を自分で実装してみましょう。

### アルゴリズムの仕組み
1. 元の行列 $A$ の右側に単位行列 $I$ を並べた「拡大行列」 $[A | I]$ を作ります。
2. 行基本変形（行を何倍かする、行同士を足し引きする）を繰り返し、左側の $A$ を単位行列 $I$ に変形させます。
3. このとき、同時に右側の $I$ に行っていた操作の結果、右側に残った行列が $A^{-1}$ になります。

$$[A | I] \xrightarrow{\text{変形}} [I | A^{-1}]$$

次のセルで、このステップを忠実に Python コードに落とし込んでいきます。

In [2]:
import numpy as np

def manual_inverse(matrix):
    # 行列のサイズを取得（n x n）
    n = matrix.shape[0]
    
    # 1. 単位行列を作成し、元の行列の右側に結合して拡大行列を作る
    # float型にしておかないと計算過程で整数丸めが発生するため注意
    identity = np.eye(n)
    augmented = np.hstack((matrix.astype(float), identity))
    
    # 2. 各列を軸（ピボット）として掃き出しを行う
    for i in range(n):
        # 対角成分を1にする（その行全体を対角成分で割る）
        pivot = augmented[i, i]
        if pivot == 0:
            raise ValueError("この行列は正則ではないため、逆行列を持ちません。")
        
        augmented[i] = augmented[i] / pivot
        
        # 他の行のその列の成分を0にする
        for j in range(n):
            if i != j:
                factor = augmented[j, i]
                augmented[j] -= factor * augmented[i]
    
    # 3. 右半分の n x n 部分が逆行列
    inv_matrix = augmented[:, n:]
    return inv_matrix

# --- テスト実行 ---
A = np.array([[1, 2],
              [3, 4]])

A_inv = manual_inverse(A)

print("元の行列 A:\n", A)
print("手作り関数で求めた逆行列 A_inv:\n", A_inv)
print("検証 (A @ A_inv):\n", np.round(A @ A_inv, 10)) # 単位行列になれば成功

元の行列 A:
 [[1 2]
 [3 4]]
手作り関数で求めた逆行列 A_inv:
 [[-2.   1. ]
 [ 1.5 -0.5]]
検証 (A @ A_inv):
 [[1. 0.]
 [0. 1.]]


In [3]:
def inverse_2x2(A):
    a, b, c, d = A.flatten()
    det = a*d - b*c
    if det == 0:
        return None
    return (1/det) * np.array([[d, -b], [-c, a]])

In [4]:
import numpy as np

def create_square_matrix(n, mode='int', low=1, high=10):
    """
    指定したサイズの正方行列を生成する
    mode='int'  : lowからhigh-1までの整数
    mode='float': 0.0から1.0までの実数
    """
    if mode == 'int':
        # 整数で生成（手計算の確認に便利）
        return np.random.randint(low, high, size=(n, n))
    else:
        # 0〜1の実数で生成
        return np.random.rand(n, n)

# --- 使用例 ---
# 3x3の整数行列を作る
A_random = create_square_matrix(3, mode='int')
print("生成された行列 A_random:")
print(A_random)

生成された行列 A_random:
[[7 2 2]
 [3 4 9]
 [3 2 8]]


In [5]:
A_random

array([[7, 2, 2],
       [3, 4, 9],
       [3, 2, 8]])

In [7]:
inv_A=manual_inverse(A_random)

In [12]:
print(np.round(A_random@inv_A,10)+0.0)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [13]:
np.round(inv_A,2)

array([[ 0.15, -0.13,  0.11],
       [ 0.03,  0.54, -0.62],
       [-0.07, -0.09,  0.24]])

In [11]:
0.0 == -0.0

True

In [20]:
print(A_random.shape)
print(A_random.shape[0])
print(A_random.size)
print(len(A_random))

(3, 3)
3
9
3


In [15]:
A_random

array([[7, 2, 2],
       [3, 4, 9],
       [3, 2, 8]])